## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов.

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать параметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор.

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import *
from sklearn.model_selection import train_test_split
import nltk
from nltk import ngrams
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv("/content/drive/MyDrive/Data/women-clothing-accessories.csv", sep = '\t', usecols=[0, 1])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data.sample(10)

,review,sentiment
9875,деньги пока не получила . шел больше месяца,negative
86101,Кофта подошла идеально по размеру. Описанию со...,positive
49940,"Рукава короткие, узкая в талии. Думала, матери...",neautral
50856,Слишком много push up'а,neautral
64754,"Очень милый свитер. и не толстый и не тонкий, ...",positive
52296,"все хорошо, но запах ! боюсь одевать",neautral
11991,продавец *удак. трек дал левый на другого чело...,negative
55491,"размер L оказался мал , еле натянула , буду на...",neautral
15153,3хл это 44 размер.мандец .на хрена такое шить?...,negative
52115,маленький размер; не соответствует размерной с...,neautral


In [ ]:
X_train, x_test, y_train, y_test = train_test_split(data.review, data.sentiment, train_size = 0.7)
y_train.value_counts()

neautral    21014
negative    21007
positive    20978
Name: sentiment, dtype: int64

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 1))
vectorized_x_train = vectorizer.fit_transform(X_train)
list(vectorizer.vocabulary_.items())[:10]

[('нормальная', 22046),
 ('обычная', 23145),
 ('водолазка', 7247),
 ('продавец', 31866),
 ('супер', 38937),
 ('заказ', 12507),
 ('пришел', 31557),
 ('через', 43303),
 ('14', 272),
 ('дней', 10584)]

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)
vectorized_x_test = vectorizer.transform(x_test)
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    neautral       0.59      0.68      0.63      8986
    negative       0.73      0.63      0.68      8993
    positive       0.85      0.84      0.85      9022

    accuracy                           0.72     27001
   macro avg       0.72      0.72      0.72     27001
weighted avg       0.72      0.72      0.72     27001



In [ ]:
!pip install -q pymorphy2

In [ ]:
#предобработка текста

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
import string
pymorph = MorphAnalyzer()
# noise = stopwords.words('russian')
# smart_vectorizer = CountVectorizer(ngram_range=(1, 1), stop_words=noise, lowercase=True)
# smart_vectorized_x_train = smart_vectorizer.fit_transform(X_train)
for i in range(data.shape[0] - 1):
  row = data.iloc[i].review
  new_row = ""

  for ch in string.punctuation:
    row = row.replace(ch,"") # удаление этого !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

  tokens = word_tokenize(row)
  for cur_token in tokens:
    normf = pymorph.parse(cur_token)
    new_row = new_row + normf[0].normal_form + " "
  new_row = new_row[:-1]
  data.iloc[i].review = new_row


In [ ]:
data.sample(10)

,review,sentiment
9643,заказать майка 5 май сегодня 22 август заказ т...,negative
38186,на рост 156 вес 55 маленький очень короткий,neautral
83542,маленький на 46 xl,positive
44518,рисунок на вещь не совпадать рукав широкий,neautral
78028,колготки отличный качество я очень понравиться...,positive
69951,хороший джинсы такой план заказывать уже 2ой р...,positive
78600,свитер прекрасный тёплый трек отслеживаться бы...,positive
57036,доставка быстрый качество не устроить катышек ...,neautral
83764,платье классный прийти быстро очень понравитьс...,positive
8726,товар так и не поступить продавец просить закр...,negative


In [ ]:
X_train, x_test, y_train, y_test = train_test_split(data.review, data.sentiment, train_size = 0.7)
noise = stopwords.words('russian')
smart_vectorizer = CountVectorizer(ngram_range=(1, 1), stop_words=noise, lowercase=True)
smart_vectorized_x_train = smart_vectorizer.fit_transform(X_train)
print(smart_vectorized_x_train.shape)
clf = MultinomialNB()
clf.fit(smart_vectorized_x_train, y_train)
smart_vectorized_x_test = smart_vectorizer.transform(x_test)
pred = clf.predict(smart_vectorized_x_test)
print(classification_report(y_test, pred))

(62999, 52010)
              precision    recall  f1-score   support

    neautral       0.59      0.64      0.61      9052
    negative       0.71      0.63      0.67      8957
    positive       0.81      0.83      0.82      8992

    accuracy                           0.70     27001
   macro avg       0.70      0.70      0.70     27001
weighted avg       0.70      0.70      0.70     27001



In [ ]:
smart_vectorizer_2gram = CountVectorizer(ngram_range=(1, 2), stop_words=noise, lowercase=True)
smart_vectorized_x_train = smart_vectorizer_2gram.fit_transform(X_train)
print(smart_vectorized_x_train.shape)
clf = MultinomialNB()
clf.fit(smart_vectorized_x_train, y_train)
smart_vectorized_x_test = smart_vectorizer_2gram.transform(x_test)
pred = clf.predict(smart_vectorized_x_test)
print(classification_report(y_test, pred))

(62999, 398476)
              precision    recall  f1-score   support

    neautral       0.60      0.63      0.62      9052
    negative       0.71      0.66      0.68      8957
    positive       0.83      0.85      0.84      8992

    accuracy                           0.71     27001
   macro avg       0.71      0.71      0.71     27001
weighted avg       0.71      0.71      0.71     27001



In [ ]:
smart_vectorizer_3gram = CountVectorizer(ngram_range=(1, 3), stop_words=noise, lowercase=True)
smart_vectorized_x_train = smart_vectorizer_3gram.fit_transform(X_train)
print(smart_vectorized_x_train.shape)
clf = MultinomialNB()
clf.fit(smart_vectorized_x_train, y_train)
smart_vectorized_x_test = smart_vectorizer_3gram.transform(x_test)
pred = clf.predict(smart_vectorized_x_test)
print(classification_report(y_test, pred))

(62999, 992025)
              precision    recall  f1-score   support

    neautral       0.61      0.61      0.61      9052
    negative       0.70      0.67      0.69      8957
    positive       0.82      0.85      0.83      8992

    accuracy                           0.71     27001
   macro avg       0.71      0.71      0.71     27001
weighted avg       0.71      0.71      0.71     27001



## TF-IDF векторизация

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1), lowercase=True)
tfidf_vectorized_x_train = tfidf_vectorizer.fit_transform(X_train)
clf = MultinomialNB()
clf.fit(tfidf_vectorized_x_train, y_train)
tfidf_vectorized_x_test = tfidf_vectorizer.transform(x_test)
pred = clf.predict(tfidf_vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    neautral       0.59      0.67      0.63      9052
    negative       0.71      0.64      0.67      8957
    positive       0.85      0.82      0.83      8992

    accuracy                           0.71     27001
   macro avg       0.72      0.71      0.71     27001
weighted avg       0.72      0.71      0.71     27001



In [ ]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), lowercase=True)
tfidf_vectorized_x_train = tfidf_vectorizer.fit_transform(X_train)
clf = MultinomialNB()
clf.fit(tfidf_vectorized_x_train, y_train)
tfidf_vectorized_x_test = tfidf_vectorizer.transform(x_test)
pred = clf.predict(tfidf_vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    neautral       0.62      0.68      0.65      9052
    negative       0.72      0.67      0.69      8957
    positive       0.88      0.85      0.86      8992

    accuracy                           0.73     27001
   macro avg       0.74      0.73      0.73     27001
weighted avg       0.74      0.73      0.73     27001



In [ ]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), lowercase=True, max_df=0.9, min_df=0.3, max_features=10000)
tfidf_vectorized_x_train = tfidf_vectorizer.fit_transform(X_train)
clf = MultinomialNB()
clf.fit(tfidf_vectorized_x_train, y_train)
tfidf_vectorized_x_test = tfidf_vectorizer.transform(x_test)
pred = clf.predict(tfidf_vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    neautral       0.38      0.06      0.10      9052
    negative       0.37      0.70      0.48      8957
    positive       0.41      0.39      0.40      8992

    accuracy                           0.38     27001
   macro avg       0.39      0.38      0.33     27001
weighted avg       0.39      0.38      0.33     27001



In [ ]:
#Символьные n-граммы
smart_vectorizer_2gram = CountVectorizer(ngram_range=(2, 3), stop_words=noise, lowercase=True, analyzer='char')
smart_vectorized_x_train = smart_vectorizer_2gram.fit_transform(X_train)
print(smart_vectorized_x_train.shape)
clf = MultinomialNB()
clf.fit(smart_vectorized_x_train, y_train)
smart_vectorized_x_test = smart_vectorizer_2gram.transform(x_test)
pred = clf.predict(smart_vectorized_x_test)
print(classification_report(y_test, pred))

(62999, 27770)
              precision    recall  f1-score   support

    neautral       0.56      0.67      0.61      9052
    negative       0.70      0.59      0.64      8957
    positive       0.82      0.79      0.81      8992

    accuracy                           0.68     27001
   macro avg       0.70      0.68      0.69     27001
weighted avg       0.70      0.68      0.69     27001



## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах.

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **?а** - ноль или один символ **а**
* **+а** - один или более символов **а**
* **\*а** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа

Пример:
Выражению \*a?b. соответствуют последовательности a, ab, abc, aa, aac НО НЕ abb!

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**:
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc')
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
text = "I won one Titled Tuesday this Oct with some very decent black opening play against Peter Svidler and Andrew Tang."
print(re.findall(r'\b[A-Za-z]\S', text))

['wo', 'on', 'Ti', 'Tu', 'th', 'Oc', 'wi', 'so', 've', 'de', 'bl', 'op', 'pl', 'ag', 'Pe', 'Sv', 'an', 'An', 'Ta']


### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie')
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=1)
print(result)

['itsy', ' bitsy, teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
text = "I won one Titled Tuesday this Oct with some very decent black opening play against Peter Svidler and Andrew Tang. Both were critical games for winning this Titled Tuesday.  Here's some “blitz” commentary for these 3|1 games."
list = re.split('\.', text, maxsplit=2)
print(list)

['I won one Titled Tuesday this Oct with some very decent black opening play against Peter Svidler and Andrew Tang', ' Both were critical games for winning this Titled Tuesday', "  Here's some “blitz” commentary for these 3|1 games."]


### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
text = "2ITAL November"
print(re.sub('\d', 'DIG', text))

DIGITAL November


**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
text = "2021ITAL November https://vk.com/al_feed.php December"
print(re.sub(r'https://[^\s]+', '', text))

2021ITAL November  December


### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
prog = re.compile(r'[А-Яа-яё\-]{4,}')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'больше', 'больше', 'слов', 'Что-то']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
import re
prog = re.compile(r'@[a-zA-Z]+\.[a-zA-Z]{2,5}\b')
prog.findall("abc.test@gmail.comkdjfhvdfiv, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz")

['@test.in', '@analyticsvidhya.com', '@rest.biz']

In [ ]:
print(re.sub(r'\b\d{1,2} [A-Z][a-z]{1,} ', '', '1 April 1966'))

1966
